# 如何保存Keras模型？

## 保存/加载整个模型（结构＋权重＋优化器状态）

不建议使用pickle或cPickle来保存Keras模型。
你可以使用model.save(filepath)将Kreas模型保存到单个HDF5文件中，该文件将包含:

* 模型的结构，允许重新创建模型
* 模型的权重
* 训练配置项（损失函数，优化器）
* 优化器状态，允许准确地从你上次结束的地方继续训练

你可以使用keras.models.load_model(filepath)重新实例化模型。load_model还将负责使用保存的训练配置项来编译模型（除非模型从未编译过）。

In [5]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential()
model.add(Dense(100, input_shape=(1000, 10)))
model.add(Activation('relu'))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(20, activation='sigmoid'))

model.save("my_model.h5") #创建ＨＤＦ５文件“my_model.h5"
del model # 删除现有模型

# 返回一个编译好的模型
# 与之前那个相同
model = load_model("my_model.h5")

/home/hanwei/soft/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


## 只保存/加载模型的结构
如果您只需保存**模型的结构**, 而非其权重或者训练配置项，则可以执行一下操作：

In [9]:
# 保存为JSON
json_string = model.to_json()

# 保存为YAML
yaml_string = model.to_yaml()

生成的JSON/YAML文件是人类可读的，如果需要还可以手动编辑
你可以从这些模型建立一个新的模型：

In [10]:
# 从JSON重建模型
from keras.models import model_from_json

model = model_from_json(json_string)

# 从YAML重建模型：
from keras.models import model_from_yaml
model = model_from_yaml(yaml_string)

## 只保存/加载模型权重
如果您只需要**模型的权重**，可以使用下面的代码以HDF5格式进行保存。
请注意，我们首先需要安装HDF5和Python库h5py，它们不包括在keras中。

In [11]:
model.save_weights("my_model_weights.h5")

假设你有用于实例化模型的代码，则可以将保存的权重加载到具有相同结构的模型中：

In [12]:
model.load_weights("my_model_weights.h5")

如果你需要将权重加载到不同的结构（有一些共同层）的模型中，例如微调或迁移学习，则可以按层的名字来加载权重：

In [13]:
model.load_weights("my_model_weights.h5", by_name=True)

In [22]:
"""
假设原始模型入下所示：
model = Sequential()
model.add(Dense(2, input_dim=3, name='dense_1'))
model.add(Dense(3, name='dense_2'))
...
model.save_weights(fname) mkjik
model = Sequential()
model.add(Dense(2, input_dim=3, name='dense_1'))  # 将被加载
model.add(Dense(10, name='new_dense'))  # 将不被加载

# 从第一个模型加载权重；只会影响第一层，dense_1
# model.load_weights(fname, by_name=True)

## 处理已保存模型中的自定义层（或其他自定义对象）

如果要加载的模型包含自定义层或其他自定义层或函数，则可以通过custom_objects参数将它们传递给加载机制：

In [24]:
from keras.models import load_model

#　假设你的模型包含一个AttentionLayer 类的实例
# model = load_model('my_model.h5', custom_objects={'AttentionLayer': AttentionLayer})

或者，你可以使用自定义对象的作用域：

In [25]:
from keras.utils import CustomObjectScope

In [28]:
# with CustomObjectScope({'AttentionLayer':AttentionLayer}):
#     model = load_model('my_model.h5')

自定义对象的处理与 load_model, model_from_json, model_from_yaml 的工作方式相同：

In [30]:
from keras.models import model_from_json
# model = model_from_json(json_string, custom_objects={'AttentionLayer': AttentionLayer})